1.导入库

In [16]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

2.变量初始化

In [17]:
# 加载数据

# 被解释变量
house_price = pd.read_csv('商品建筑销售价格年初至今平均住宅.csv')
# 解释变量
house_sale_area = pd.read_csv('房屋销售面积35个城市住宅.csv') 
income = pd.read_csv('人均可支配收入.csv')
all_populations = pd.read_csv('人口.csv')
urban_rate  = pd.read_csv('人口数城镇化率常住.csv')
house_tax = pd.read_csv('财政收入一般公共预算收入税收房产税.csv')

In [30]:
# 列表变量定义
x = [i for i in range(2007, 2024)] # x轴年份 2007-2023

In [29]:
# 房价
y_house_price_month = []
y_house_price_year = []

start_row_house_price = 134
count = 0
for index, row in house_price.iloc[start_row_house_price:].iterrows():
    y_house_price_month.append(float(row[1]))
    count += 1
    if count % 12 == 0:
        y_house_price_year.append(sum(y_house_price_month) / 12)
        y_house_price_month = []


len(y_house_price_year)

C:\Users\25337\AppData\Local\Temp\ipykernel_14352\2318963108.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_house_price_month.append(float(row[1]))


17

In [32]:
# 变量读取
y_house_sale_area = []
y_income = []
y_house_tax = []

start_row_house_sale_area = 36
start_row_income = 59
start_row_house_tax = 28

for index, row in house_sale_area.iloc[start_row_house_sale_area:].iterrows():
    y_house_sale_area.append(float(row[1]))

for index, row in income.iloc[start_row_income:].iterrows():
    y_income.append(float(row[1]))

for index, row in house_tax.iloc[start_row_house_tax:].iterrows():
    y_house_tax.append(float(row[1]))

len(y_house_sale_area), len(y_income), len(y_house_tax)

C:\Users\25337\AppData\Local\Temp\ipykernel_14352\742234643.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_house_sale_area.append(float(row[1]))
C:\Users\25337\AppData\Local\Temp\ipykernel_14352\742234643.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_income.append(float(row[1]))
C:\Users\25337\AppData\Local\Temp\ipykernel_14352\742234643.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_house_tax.append(float(r

(17, 17, 17)

In [25]:
# 城镇人口
y_all_populations = []
y_urban_rate = []
y_urban_populations = []

start_row_population = 86
start_row_urban_rate = 53
for index, row in all_populations.iloc[start_row_population:].iterrows():
    y_all_populations.append(float(row[1]))
for index, row in urban_rate.iloc[start_row_urban_rate:].iterrows():
    y_urban_rate.append(float(row[1]))

y_urban_populations = [a*b/100 for a, b in zip(y_all_populations, y_urban_rate)]

C:\Users\25337\AppData\Local\Temp\ipykernel_14352\1340348645.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_all_populations.append(float(row[1]))
C:\Users\25337\AppData\Local\Temp\ipykernel_14352\1340348645.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_urban_rate.append(float(row[1]))


3.数据预处理

In [34]:
# 归一化
y_house_price_year = [(i - min(y_house_price_year)) / (max(y_house_price_year) - min(y_house_price_year)) for i in y_house_price_year]

y_house_sale_area = [(i - min(y_house_sale_area)) / (max(y_house_sale_area) - min(y_house_sale_area)) for i in y_house_sale_area]
y_income = [(i - min(y_income)) / (max(y_income) - min(y_income)) for i in y_income]
y_house_tax = [(i - min(y_house_tax)) / (max(y_house_tax) - min(y_house_tax)) for i in y_house_tax]
y_urban_populations = [(i - min(y_urban_populations)) / (max(y_urban_populations) - min(y_urban_populations)) for i in y_urban_populations]

y_house_price_year

[0.0,
 0.013712274616718798,
 0.11040783305859986,
 0.17658323866774964,
 0.23475392965879835,
 0.2523085360273773,
 0.35136252272393836,
 0.3397214138018232,
 0.39267567973695394,
 0.5137920075942902,
 0.5442656889343184,
 0.6648243588792179,
 0.7710233375788736,
 0.8487933806174357,
 0.9858726662182713,
 0.8896185869570693,
 1.0]

4.灰色关联度分析

In [35]:
# 灰色关联度分析函数定义
def grey_relational_analysis(ref_sequence, cmp_sequences, rho=0.5):
    """
    灰色关联度分析
    :param ref_sequence: 参考序列,numpy数组
    :param cmp_sequences: 比较序列的列表,每个元素是numpy数组
    :param rho: 分辨系数，通常在(0, 1)之间,默认0.5
    :return: 关联度列表
    """
    # 数据预处理 - 初始化算子（这里使用均值化）
    ref_sequence = ref_sequence / np.mean(ref_sequence)
    cmp_sequences = [seq / np.mean(seq) for seq in cmp_sequences]
    
    # 计算关联系数
    n = len(ref_sequence)
    m = len(cmp_sequences)
    grey_relational_coefficient = np.zeros((m, n))
    
    min_diff = np.inf
    max_diff = 0
    
    # 寻找最小差和最大差
    for i in range(m):
        diff = np.abs(ref_sequence - cmp_sequences[i])
        min_diff = min(min_diff, np.min(diff))
        max_diff = max(max_diff, np.max(diff))
    
    # 计算关联系数矩阵
    for i in range(m):
        diff = np.abs(ref_sequence - cmp_sequences[i])
        grey_relational_coefficient[i, :] = (min_diff + rho * max_diff) / (diff + rho * max_diff)
    
    # 计算关联度
    grey_relational_degree = np.mean(grey_relational_coefficient, axis=1)
    
    return grey_relational_degree

In [37]:
ref_sequence = np.array(y_house_price_year)
cmp_sequences = [
    np.array(y_house_sale_area),
    np.array(y_income),
    np.array(y_urban_populations),
    np.array(y_house_tax)
]

# 进行灰色关联度分析
degrees = grey_relational_analysis(ref_sequence, cmp_sequences)

# 输出关联度结果
for i, degree in enumerate(degrees):
    print(f"因素{i+1}与参考因素的关联度: {degree}")

因素1与参考因素的关联度: 0.7025651616968775
因素2与参考因素的关联度: 0.9247963127188182
因素3与参考因素的关联度: 0.8917520089923784
因素4与参考因素的关联度: 0.8455331059970925
